<a href="https://colab.research.google.com/github/AlexKressner/WS24_Supply_Chain_Optimierung/blob/main/NetworkDesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Design

In [48]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [49]:
! pip install -q pyscipopt

In [50]:
import pandas as pd
from pyscipopt import Model, quicksum

## Daten aus csv-Datei laden



In [51]:
! ls WS24_Supply_Chain_Optimierung/Daten/NetworkDesign

NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [52]:
folder = "WS24_Supply_Chain_Optimierung/Daten/NetworkDesign"

In [53]:
path = f"{folder}/NetworkDesign_Invest_Kapa.csv"

In [54]:
kapa=pd.read_csv(path, sep=";")

In [55]:
kapa.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [56]:
path = f"{folder}/NetworkDesign_Produktionskosten.csv"

In [57]:
produktionskosten=pd.read_csv(path, sep=";")

In [58]:
produktionskosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [59]:
path = f"{folder}/NetworkDesign_Nachfrage.csv"

In [60]:
nachfrage=pd.read_csv(path, sep=";")

In [61]:
nachfrage.head()

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


## Optimierungsmodell

In [62]:
scip = Model()

### Indexmengen

In [63]:
I = produktionskosten["Produktionsstandort"].unique().tolist() # Menge der Produktionsstandorte

In [64]:
I

['Charleston', 'Curitiba', 'Hamburg', 'Quanzhou', 'East London']

In [65]:
J = nachfrage["Markt"].tolist() #Menge der Märkte

**Variante 2:** Neuer Markt

In [66]:
J.append("Arabische_Halbinsel")

In [67]:
J

['USA', 'Südamerika', 'Europa', 'Asien', 'Afrika', 'Arabische_Halbinsel']

In [68]:
A = kapa["Ausbaustufe"].unique().tolist() # Menge der Ausbaustufen an einem Standort

In [69]:
A

['klein', 'groß']

### Entscheidungsvariablen

In [70]:
# Definition der Entscheidungsvariablen
# X sind die Flussvariablen, d.h. Herstell- und Transportmenge von i nach j
X={}
for i in I:
  for j in J:
    X[i,j] = scip.addVar(vtype="C", name=f"{i},{j}")

In [71]:
# Definition der Entscheidungsvariablen
# Y sind die Strukturvariablen, d.h. ob und mit welchen Kapazitäten ein Standort ausgebaut wird
Y={}
for i in I:
  for a in A:
    Y[i,a] = scip.addVar(vtype="B", name=f"{i},{a}")

In [72]:
print('Anzahl Entscheidungsvariablen =', len(scip.getVars()))

Anzahl Entscheidungsvariablen = 40


### Parameter

In [73]:
i = "Produktionsstandort"
j = "Markt"

In [74]:
# Herstell- und Transportkosten
produktionskosten.set_index([i,j], inplace=True)

In [75]:
cp = produktionskosten.to_dict("dict")["Produktionskosten"]

In [76]:
cp

{('Charleston', 'USA'): 81,
 ('Charleston', 'Südamerika'): 92,
 ('Charleston', 'Europa'): 101,
 ('Charleston', 'Asien'): 130,
 ('Charleston', 'Afrika'): 115,
 ('Curitiba', 'USA'): 117,
 ('Curitiba', 'Südamerika'): 77,
 ('Curitiba', 'Europa'): 108,
 ('Curitiba', 'Asien'): 98,
 ('Curitiba', 'Afrika'): 100,
 ('Hamburg', 'USA'): 102,
 ('Hamburg', 'Südamerika'): 105,
 ('Hamburg', 'Europa'): 95,
 ('Hamburg', 'Asien'): 119,
 ('Hamburg', 'Afrika'): 111,
 ('Quanzhou', 'USA'): 115,
 ('Quanzhou', 'Südamerika'): 125,
 ('Quanzhou', 'Europa'): 90,
 ('Quanzhou', 'Asien'): 59,
 ('Quanzhou', 'Afrika'): 74,
 ('East London', 'USA'): 142,
 ('East London', 'Südamerika'): 100,
 ('East London', 'Europa'): 103,
 ('East London', 'Asien'): 105,
 ('East London', 'Afrika'): 71}

**Variante 2:** Neuer Markt

In [77]:
cp['Charleston','Arabische_Halbinsel']=125
cp['Curitiba','Arabische_Halbinsel']=100
cp['Hamburg','Arabische_Halbinsel']=100
cp['Quanzhou','Arabische_Halbinsel']=75
cp['East London','Arabische_Halbinsel']=85

In [78]:
nachfrage

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


In [79]:
# Nachfrage aus den Märtken
nachfrage.set_index(["Markt"], inplace=True)

In [80]:
d = nachfrage.to_dict("dict")["Nachfragemenge"]

**Variante 2:** Neuer Markt

In [81]:
d["Arabische_Halbinsel"] = 5

In [82]:
d

{'USA': 12,
 'Südamerika': 8,
 'Europa': 14,
 'Asien': 16,
 'Afrika': 7,
 'Arabische_Halbinsel': 5}

In [83]:
# Kapazitäten der Ausbaustufen und Investitionskosten
kapazitaeten = kapa.set_index("Ausbaustufe")

In [84]:
cap = kapazitaeten.to_dict("dict")["Kapazitäten"]

In [85]:
cap

{'klein': 10, 'groß': 20}

In [86]:
cf = kapa.set_index(["Produktionsstandort","Ausbaustufe"]).to_dict("dict")["Investitionen"]

In [87]:
cf

{('Charleston', 'klein'): 6000,
 ('Charleston', 'groß'): 9000,
 ('Curitiba', 'klein'): 4500,
 ('Curitiba', 'groß'): 6750,
 ('Hamburg', 'klein'): 6500,
 ('Hamburg', 'groß'): 9750,
 ('Quanzhou', 'klein'): 4100,
 ('Quanzhou', 'groß'): 6150,
 ('East London', 'klein'): 4000,
 ('East London', 'groß'): 6000}

**Variante 1:** Subventionen

In [88]:
cf['Charleston','klein'] = cf['Charleston','klein']*0.8

In [89]:
cf['Charleston','groß'] = cf['Charleston','groß']*0.8

### Zielfunktion

min $K=\sum_{i,j} cp_{ij}* X_{ij} + \sum_{i,a} cf_{ia}* Y_{ia}$

In [90]:
# Maximierung der gesamten Deckungsbeiträge
scip.setObjective(quicksum(cp[i,j]*X[i,j] for i in I for j in J) + quicksum(cf[i,a]*Y[i,a] for i in I for a in A), sense="minimize")

### Nebenbedingungen

$\sum_i X_{ij} \ge d_j, ∀j$  

In [91]:
# Nachfrage in den Märkten erfüllen
for j in J:
  scip.addCons(quicksum(X[i,j] for i in I)>=d[j])

$\sum_j X_{ij} \le \sum_a cap_{a} * Y_{ia}, ∀i$

In [92]:
# Kapazitäten aufbauen und einhalten
for i in I:
  scip.addCons(quicksum(X[i,j] for j in J)<= sum(cap[a]*Y[i,a] for a in A))

**Variante 3:**

$\sum_j X_{ij} \geq m_i, \forall i$

In [93]:
I

['Charleston', 'Curitiba', 'Hamburg', 'Quanzhou', 'East London']

In [94]:
# Zusätzlicher Parameter, Mindestbeschäftigung
m = {}
m["Charleston"] = 5
m["Curitiba"] = 0
m["Hamburg"] = 25
m["Quanzhou"] = 0
m["East London"]=0

In [95]:
# Zusätzliche Nebenbedingung
for i in I:
  scip.addCons(quicksum(X[i,j] for j in J)>=m[i])

## Optimierung

In [96]:
scip.optimize()
print(scip.getStatus())

optimal


In [97]:
print('LÖSUNG:')
print('Zielfunktionswert (Kosten) =', scip.getObjVal())
print("\n")
print("PRODUKTIONS-MARKT-ZUTEILUNG")
for j in J:
  print(f"{j}:")
  for i in I:
    if scip.getVal(X[i,j])>0:
      print('\t', f'{i,j} =', round(scip.getVal(X[i,j])))
print("\n")
print("INVESTITIONEN")
for i in I:
  for a in A:
    if scip.getVal(Y[i,a])>0:
      print(f'{i,a} =', round(scip.getVal(Y[i,a])))

LÖSUNG:
Zielfunktionswert (Kosten) = 34750.0


PRODUKTIONS-MARKT-ZUTEILUNG
USA:
	 ('Charleston', 'USA') = 12
Südamerika:
	 ('Charleston', 'Südamerika') = 5
	 ('Hamburg', 'Südamerika') = 3
Europa:
	 ('Hamburg', 'Europa') = 14
Asien:
	 ('Hamburg', 'Asien') = 0
	 ('Quanzhou', 'Asien') = 16
Afrika:
	 ('Hamburg', 'Afrika') = 3
	 ('Quanzhou', 'Afrika') = 4
	 ('East London', 'Afrika') = 0
Arabische_Halbinsel:
	 ('Hamburg', 'Arabische_Halbinsel') = 5


INVESTITIONEN
('Charleston', 'groß') = 1
('Hamburg', 'klein') = 1
('Hamburg', 'groß') = 1
('Quanzhou', 'groß') = 1
